# SQLAlchemy ORM

Comme précédemment, on aura besoin d'un *engine* pour parler à la base de données :

In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)

## Créer des tables

Cette fois-ci on ne va pas déclarer les tables explicitements, mais les générer automatiquement à partir des classes Python correspondantes :

In [2]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Fruit(Base):
    __tablename__ = 'fruits'

    id = Column(Integer, primary_key=True)
    nom = Column(String)
    
    def __repr__(self):
        return f"<Fruit(nom={self.nom})>"

On peut retrouver la définition de la table :

In [3]:
Fruit.__table__

Table('fruits', MetaData(bind=None), Column('id', Integer(), table=<fruits>, primary_key=True, nullable=False), Column('nom', String(), table=<fruits>), schema=None)

Et pour créer les tables en base, comme précédemment :

In [4]:
Base.metadata.create_all(engine)

2019-12-10 17:40:06,413 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-10 17:40:06,415 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,416 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-10 17:40:06,416 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,417 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("fruits")
2019-12-10 17:40:06,418 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,419 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("fruits")
2019-12-10 17:40:06,420 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,421 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE fruits (
	id INTEGER NOT NULL, 
	nom VARCHAR, 
	PRIMARY KEY (id)
)


2019-12-10 17:40:06,421 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,422 INFO sqlalchemy.engine.base.Engine COMMIT


## L'élément clé : la session

Pour gérer la correspondance entre les lignes des tables dans la base et les objets Python, on va avoir besoin d'un objet qu'on appelle la **session**.

In [5]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)  # étape 1 : créer une « session factory »

db_session = Session()  # étape 2 : instancier une session

## Insérer des données

In [6]:
for nom in ["pomme", "banane", "kiwi"]:
    fruit = Fruit(nom=nom)
    db_session.add(fruit)  # la session enregistre cet objet comme étant nouveau

In [7]:
db_session.commit()  # les nouveau objets sont insérés en base

2019-12-10 17:40:06,438 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-10 17:40:06,440 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-10 17:40:06,440 INFO sqlalchemy.engine.base.Engine ('pomme',)
2019-12-10 17:40:06,441 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-10 17:40:06,442 INFO sqlalchemy.engine.base.Engine ('banane',)
2019-12-10 17:40:06,442 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-10 17:40:06,442 INFO sqlalchemy.engine.base.Engine ('kiwi',)
2019-12-10 17:40:06,443 INFO sqlalchemy.engine.base.Engine COMMIT


## Récupérer des données

In [8]:
pomme = db_session.query(Fruit).filter_by(nom="pomme").one()
hex(id(pomme))

2019-12-10 17:40:06,449 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-10 17:40:06,450 INFO sqlalchemy.engine.base.Engine SELECT fruits.id AS fruits_id, fruits.nom AS fruits_nom 
FROM fruits 
WHERE fruits.nom = ?
2019-12-10 17:40:06,450 INFO sqlalchemy.engine.base.Engine ('pomme',)


'0x1f266b4c9c8'

In [9]:
pomme = db_session.query(Fruit).filter_by(nom="pomme").one()
hex(id(pomme))

2019-12-10 17:40:06,458 INFO sqlalchemy.engine.base.Engine SELECT fruits.id AS fruits_id, fruits.nom AS fruits_nom 
FROM fruits 
WHERE fruits.nom = ?
2019-12-10 17:40:06,459 INFO sqlalchemy.engine.base.Engine ('pomme',)


'0x1f266b4c9c8'

In [10]:
banane = db_session.query(Fruit).filter_by(nom="banane").one()
banane

2019-12-10 17:40:06,468 INFO sqlalchemy.engine.base.Engine SELECT fruits.id AS fruits_id, fruits.nom AS fruits_nom 
FROM fruits 
WHERE fruits.nom = ?
2019-12-10 17:40:06,468 INFO sqlalchemy.engine.base.Engine ('banane',)


<Fruit(nom=banane)>

In [11]:
db_session.query(Fruit).all()

2019-12-10 17:40:06,477 INFO sqlalchemy.engine.base.Engine SELECT fruits.id AS fruits_id, fruits.nom AS fruits_nom 
FROM fruits
2019-12-10 17:40:06,478 INFO sqlalchemy.engine.base.Engine ()


[<Fruit(nom=pomme)>, <Fruit(nom=banane)>, <Fruit(nom=kiwi)>]

In [12]:
db_session.query(Fruit.id, Fruit.nom).filter_by(nom="pomme").one()

2019-12-10 17:40:06,485 INFO sqlalchemy.engine.base.Engine SELECT fruits.id AS fruits_id, fruits.nom AS fruits_nom 
FROM fruits 
WHERE fruits.nom = ?
2019-12-10 17:40:06,486 INFO sqlalchemy.engine.base.Engine ('pomme',)


(1, 'pomme')

## Modifier des données

In [13]:
pomme.nom = "poire"

In [14]:
db_session.dirty

IdentitySet([<Fruit(nom=poire)>])

In [15]:
banane.nom = "ananas"

In [16]:
db_session.dirty

IdentitySet([<Fruit(nom=poire)>, <Fruit(nom=ananas)>])

In [17]:
db_session.commit()

2019-12-10 17:40:06,517 INFO sqlalchemy.engine.base.Engine UPDATE fruits SET nom=? WHERE fruits.id = ?
2019-12-10 17:40:06,517 INFO sqlalchemy.engine.base.Engine (('poire', 1), ('ananas', 2))
2019-12-10 17:40:06,518 INFO sqlalchemy.engine.base.Engine COMMIT


## Relations

Ajoutons une autre classe, et une relation avec la première :

In [18]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Barquette(Base):
    __tablename__ = "barquettes"

    id = Column(Integer, primary_key=True)
    fruit_id = Column(Integer, ForeignKey("fruits.id"), nullable=False)  # clé étrangère
    poids = Column(Integer, nullable=False)

    fruit = relationship(Fruit)  # chargé à la demande
    fruit = relationship(Fruit, lazy="joined")  # toujours pré-chargé avec un join

Base.metadata.create_all(engine)

2019-12-10 17:40:06,527 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("fruits")
2019-12-10 17:40:06,527 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,529 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("barquettes")
2019-12-10 17:40:06,529 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,530 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("barquettes")
2019-12-10 17:40:06,530 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,531 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE barquettes (
	id INTEGER NOT NULL, 
	fruit_id INTEGER NOT NULL, 
	poids INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(fruit_id) REFERENCES fruits (id)
)


2019-12-10 17:40:06,531 INFO sqlalchemy.engine.base.Engine ()
2019-12-10 17:40:06,532 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
kaki = Fruit(nom="kaki")
barquette = Barquette(fruit=kaki, poids=500)

In [20]:
db_session.add(barquette)

In [21]:
db_session.commit()

2019-12-10 17:40:06,549 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-10 17:40:06,550 INFO sqlalchemy.engine.base.Engine INSERT INTO fruits (nom) VALUES (?)
2019-12-10 17:40:06,550 INFO sqlalchemy.engine.base.Engine ('kaki',)
2019-12-10 17:40:06,551 INFO sqlalchemy.engine.base.Engine INSERT INTO barquettes (fruit_id, poids) VALUES (?, ?)
2019-12-10 17:40:06,552 INFO sqlalchemy.engine.base.Engine (4, 500)
2019-12-10 17:40:06,552 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
barquette.fruit_id

2019-12-10 17:40:06,557 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-12-10 17:40:06,558 INFO sqlalchemy.engine.base.Engine SELECT barquettes.id AS barquettes_id, barquettes.fruit_id AS barquettes_fruit_id, barquettes.poids AS barquettes_poids 
FROM barquettes 
WHERE barquettes.id = ?
2019-12-10 17:40:06,559 INFO sqlalchemy.engine.base.Engine (1,)


4

In [23]:
barquette.fruit

2019-12-10 17:40:06,566 INFO sqlalchemy.engine.base.Engine SELECT fruits.id AS fruits_id, fruits.nom AS fruits_nom 
FROM fruits 
WHERE fruits.id = ?
2019-12-10 17:40:06,566 INFO sqlalchemy.engine.base.Engine (4,)


<Fruit(nom=kaki)>

In [24]:
print(f"{barquette.poids}g de {barquette.fruit.nom}")

500g de kaki


In [25]:
db_session.expunge_all()
for barquette in db_session.query(Barquette):
    print(f"{barquette.poids}g de {barquette.fruit.nom}")

2019-12-10 17:40:06,580 INFO sqlalchemy.engine.base.Engine SELECT barquettes.id AS barquettes_id, barquettes.fruit_id AS barquettes_fruit_id, barquettes.poids AS barquettes_poids, fruits_1.id AS fruits_1_id, fruits_1.nom AS fruits_1_nom 
FROM barquettes LEFT OUTER JOIN fruits AS fruits_1 ON fruits_1.id = barquettes.fruit_id
2019-12-10 17:40:06,581 INFO sqlalchemy.engine.base.Engine ()
500g de kaki


In [26]:
from sqlalchemy.orm import joinedload

db_session.expunge_all()
for barquette in db_session.query(Barquette).options(joinedload("fruit")):
    print(f"{barquette.poids}g de {barquette.fruit.nom}")

2019-12-10 17:40:06,589 INFO sqlalchemy.engine.base.Engine SELECT barquettes.id AS barquettes_id, barquettes.fruit_id AS barquettes_fruit_id, barquettes.poids AS barquettes_poids, fruits_1.id AS fruits_1_id, fruits_1.nom AS fruits_1_nom 
FROM barquettes LEFT OUTER JOIN fruits AS fruits_1 ON fruits_1.id = barquettes.fruit_id
2019-12-10 17:40:06,590 INFO sqlalchemy.engine.base.Engine ()
500g de kaki


## Parler à plusieurs bases

In [27]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine1 = create_engine("sqlite:///base1.db")

Base1 = declarative_base()

class Fruit(Base1):
    __tablename__ = 'fruits'

    id = Column(Integer, primary_key=True)
    nom = Column(String)
    
Base1.metadata.create_all(engine1)

Session1 = sessionmaker(bind=engine1)

db_session1 = Session1()

In [28]:
engine2 = create_engine("sqlite:///base2.db")

Base2 = declarative_base()

class Client(Base2):
    __tablename__ = 'clients'

    id = Column(Integer, primary_key=True)
    nom = Column(String)
    
Base2.metadata.create_all(engine2)

Session2 = sessionmaker(bind=engine2)

db_session2 = Session2()

In [29]:
db_session1.add(Fruit(nom="banane"))
db_session1.commit()

In [30]:
for fruit in db_session1.query(Fruit):
    db_session2.add(Client(nom=f"Le vendeur de {fruit.nom}"))

In [31]:
db_session2.commit()

In [32]:
[c.nom for c in db_session2.query(Client)]

['Toto',
 'Toto',
 'Le vendeur de banane',
 'Le vendeur de banane',
 'Le vendeur de banane',
 'Le vendeur de banane',
 'Le vendeur de banane']

## Réflexion

In [33]:
engine = create_engine("sqlite:///base1.db")

In [34]:
from sqlalchemy import MetaData

meta = MetaData()

In [35]:
meta.reflect(engine)

In [36]:
meta.tables

immutabledict({'fruits': Table('fruits', MetaData(bind=None), Column('id', INTEGER(), table=<fruits>, primary_key=True, nullable=False), Column('nom', VARCHAR(), table=<fruits>), schema=None)})

In [37]:
fruits_table = meta.tables["fruits"]

In [38]:
from sqlalchemy import select

conn = engine.connect()
conn.execute(select([fruits_table.c.nom])).fetchall()

[('banane',), ('banane',), ('banane',)]

## Références

- https://docs.sqlalchemy.org/en/13/orm/tutorial.html
- https://www.martinfowler.com/eaaCatalog/unitOfWork.html
- https://learning.oreilly.com/library/view/essential-sqlalchemy-2nd/9781491916544/